In [1]:
from top2vec import Top2Vec

/home/hatma/anaconda3/envs/top2vec/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dask import dataframe as dd
import pandas as pd
import json
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

filetype = "json"
inputfile = "mixed.json"
verbose = True

if filetype == "csv":
    # tmp = pd.read_csv(inputfile, header=None, low_memory=False)
    tmp = dd.read_csv(inputfile, sep=';', header=None)
    tmp2 = tmp.to_dask_array(lengths=True)
    tmp3 = tmp2.compute()
    tmp4 = pd.DataFrame(tmp3)

    if verbose == True:
        print("\n\n csv file (before preprocessing) = ")
        print(tmp4)

    corpus = tmp4[0].str.lower().apply(remove_stopwords).to_numpy()

elif filetype == "json":
    with open(inputfile) as json_file:
        jsonfile = json.load(json_file)

    tmp = pd.DataFrame(jsonfile['results'])

    if verbose == True:
        print("\n\n json file (before preprocessing) = ")
        print(tmp)

    tmp['td'] = tmp['t'] + ' ' + tmp['d']
    corpus = tmp['td'].str.lower().apply(remove_stopwords).to_numpy()

else:
    print("ERROR, only accept csv or json file!")




 json file (before preprocessing) = 
                                                     t  \
0                             Movie Coverage - YouTube   
1       Marvel Studios Celebrates The Movies - YouTube   
2                       FRESH Movie Trailers - YouTube   
3                           Movie: The Movie - YouTube   
4    Tippu Hindi Dubbed Full Movie | Satya Karthik,...   
..                                                 ...   
193  Amazon Music Unlimited | 70 million songs ad-f...   
194                  Music Ally Is A Knowledge Company   
195                              Music - Rolling Stone   
196  MusicRadar: Music Gear, Equipment, News, Tutor...   
197                Get started | Learning Music (Beta)   

                                                     a  \
0    https://www.youtube.com/channel/UCwYzZs_hwA6Nd...   
1      https://www.youtube.com/watch%3Fv%3DQdpxoFcdORI   
2         https://www.youtube.com/c/FreshMovieTrailers   
3      https://www.youtube.com/w

In [3]:
model = Top2Vec(documents=corpus.tolist() )

2022-08-21 16:20:19,662 - top2vec - INFO - Pre-processing documents for training
2022-08-21 16:20:19,675 - top2vec - INFO - Creating joint document/word embedding


validate documents
198
<class 'list'>


2022-08-21 16:20:19,947 - top2vec - INFO - Creating lower dimension embedding of documents
2022-08-21 16:20:25,762 - top2vec - INFO - Finding dense areas of documents
2022-08-21 16:20:25,767 - top2vec - INFO - Finding topics


ValueError: need at least one array to concatenate

In [ ]:
model.get_num_topics()

In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()

print(topic_sizes)
print(topic_nums)


In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(2)

print(topic_words)
print(word_scores)
print(topic_nums)

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["music"], num_topics=2)

print(topic_words)
print(word_scores)
print(topic_scores)
print(topic_nums)

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["music"], num_topics=2)
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=2)

print(documents)
print(document_scores)
print(document_ids)

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=2)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["music", "youtube"], num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

In [ ]:
words, word_scores = model.similar_words(keywords=["music"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")